<a href="https://colab.research.google.com/github/Santapaji/ARORA/blob/main/TASK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [177]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

# Load data
data = pd.read_csv("/content/sample_data/data.csv")
labels = pd.read_csv("/content/sample_data/label.csv")

# Extract ACCX data
accx_data = data['ACC X'].values
#accx_data = accx_data.to_numpy()

def windowed_view(arr, window_size, step_size):
    """Return a windowed view of a 1D array."""
    num_windows = ((arr.size - window_size) // step_size) + 1
    return np.lib.stride_tricks.sliding_window_view(arr, window_size)[::step_size]

# Split accx_data into overlapping windows of size 1024 (for example)
window_size = 1024
step_size = 512  # 50% overlap
windows = windowed_view(accx_data, window_size, step_size)

# Convert each window to a spectrogram
spectrograms = [convert_to_spectrogram(w) for w in windows]






/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(


In [ ]:
print(spectrograms)


In [179]:
import numpy as np

# Assuming `spectrograms` is a list or array where each element is a spectrogram
example_spectrogram = spectrograms[0]
height, width = example_spectrogram.shape[:2]

# Check if it has a channel dimension
channels = 1 if len(example_spectrogram.shape) == 2 else example_spectrogram.shape[2]

print(f"Height: {height}, Width: {width}, Channels: {channels}")


Height: 1025, Width: 3, Channels: 1


In [180]:
# Install Google Cloud SDK
!curl https://sdk.cloud.google.com | bash


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0   4709      0 --:--:-- --:--:-- --:--:--  4763
######################################################################## 100.0%
Running install script from: /tmp/tmp.6ijhpeTuwR/install_google_cloud_sdk.bash
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
######################################################################## 100.0%

mkdir -p /root
"/root/google-cloud-sdk" already exists and may contain out of date files.
Remove /root/google-cloud-sdk or select a new installation directory, then run again.


In [13]:
from google.colab import auth
auth.authenticate_user()


In [140]:
!mkdir ./simclr_saved_model
!gsutil -m cp -r gs://simclr-checkpoints-tf2/simclrv2/pretrained/r101_1x_sk0/saved_model/* ./simclr_saved_model/


Copying gs://simclr-checkpoints-tf2/simclrv2/pretrained/r101_1x_sk0/saved_model/variables/variables.data-00000-of-00001...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://simclr-checkpoints-tf2/simclrv2/pretrained/r101_1x_sk0/saved_model/variables/variables.index...
Copying gs://simclr-checkpoints-tf2/simclrv2/pretrained/r101_1x_sk0/saved_model/saved_model.pb...
- [3/3 files][213.1 MiB/213.1 MiB] 100% Done                                    
Operation completed over 3 objects/213.1 MiB.                                    


In [181]:
tensor_spectrograms = tf.convert_to_tensor(spectrograms)


In [182]:
print(tensor_spectrograms.shape)
print(tensor_spectrograms.dtype)


(267, 1025, 3)
<dtype: 'float64'>


In [183]:
spectrogram_dataset = tf.data.Dataset.from_tensor_slices(spectrograms).shuffle(1024)
#augmented_dataset = spectrogram_dataset.map(augment_fn, output_signature=tf.TensorSpec(shape=(128, 128), dtype=tf.float32)).batch(32)


In [184]:
print(spectrogram_dataset)

<_ShuffleDataset element_spec=TensorSpec(shape=(1025, 3), dtype=tf.float64, name=None)>


In [185]:
# Load trained encoder (assumed from SimCLR training)
encoder = tf.saved_model.load("/content/simclr_saved_model")



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense

# Load the SimCLR-trained encoder model
encoder = tf.saved_model.load("/content/simclr_saved_model")

# Load the label dataset (label.csv)
labels_df = pd.read_csv("/content/sample_data/label.csv")
labels = labels_df['labels'].values  # Assuming 'label' is the column name

# Assuming spectrograms is a numpy array or a list
#spectrograms = ...  # Load your spectrograms data here





In [160]:
# Create a new Keras model using the loaded encoder as a layer
embedding_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Flatten()  # Flatten the output for the fully connected layers
])

# Pass spectrograms through the embedding model to get embeddings
embeddings = embedding_model.predict(spectrograms)

TypeError: ignored

In [162]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, encoder):
        super(EncoderLayer, self).__init__()
        self.encoder = encoder

    def call(self, inputs):
        return self.encoder(inputs)

# Load trained encoder (assumed from SimCLR training)
encoder = tf.saved_model.load("/content/simclr_saved_model")




ValueError: ignored

In [176]:
# Create a new Keras model using the loaded encoder as a layer
embedding_model = tf.keras.Sequential([
    EncoderLayer(encoder),
    tf.keras.layers.Flatten()  # Flatten the output for the fully connected layers
])

# Attach two fully connected layers
embedding_model.add(Dense(128, activation='relu'))
embedding_model.add(Dense(256, activation='softmax'))

# Compile and train the embedding model
embedding_model.compile(optimizer='adam', metrics=['accuracy'])
embedding_model.fit(spectrogram_dataset, epochs=10)

# Save the embedding model
embedding_model.save("/path/to/embedding_model.h5")

Epoch 1/10


TypeError: ignored